In [12]:
import en_core_web_md
import pandas as pd
from tqdm import tqdm, tqdm_pandas
import qgrid

pd.set_option('display.max_colwidth', 300)

In [13]:
tqdm.pandas()

In [14]:
nlp = en_core_web_md.load()

In [20]:
cleaned = pd.read_csv('dataset/phrases_cleaned_1522422847595.csv', usecols=['id', 'phrase'])
cleaned = cleaned.assign(doc=cleaned['phrase'].progress_apply(nlp))

100%|██████████| 287/287 [00:04<00:00, 65.25it/s]


In [21]:
SPLITS = {'who', ',','and'}
cleaned = cleaned.assign(idxs=cleaned['doc'].progress_apply(lambda doc: [tok.idx for tok in doc if tok.lemma_ in SPLITS]))
splitted_again = []
with_indicies = []
for i,row in cleaned.iterrows():
    idxs = set(row['idxs'])
    phrase = row['phrase']
    subphrases = []
    subphrase = ''
    for i,ch in enumerate(phrase):
        if i in idxs:
            subphrases.append(subphrase)
            subphrase = ch
        else:
            subphrase += ch
    splitted_again.append(subphrases)
    with_indicies.append([[tok.idx,tok] for tok in row['doc']])
        

100%|██████████| 287/287 [00:00<00:00, 30910.16it/s]


In [16]:
def add_dollars(phrase):
    doc = nlp(phrase)
    new_phrase = phrase
    acc = 0
    for tok in doc:
        if tok.lemma_ in SPLITS:
            new_phrase = new_phrase[:tok.idx+acc] + '$$' + new_phrase[tok.idx+acc:]
            acc += 2
    return new_phrase

qdf = pd.DataFrame({'text': cleaned['phrase'].apply(add_dollars)})
widget = qgrid.show_grid(qdf)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
splitted = pd.read_csv('dataset/phrases_cleaned_1522422847595.csv')
splitted.iloc[[84]]

,id,author,created,modified,search_attrs,phrase
84,98be4665-b3db-4133-a8cf-299dfa8205ab,Kostia Rybnikov,1522249286679,1522249944131,"[{""short"": ""whose bio contains 'biplane'"", ""type"": ""text"", ""name"": ""profile.bio"", ""example"": ""1""}, {""short"": ""1 times"", ""name"": ""events.count""}, {""short"": ""with positive sentiment"", ""type"": ""Sentiment"", ""name"": ""like.postSentiment"", ""human"": ""liked {sentiment} post"", ""example"": ""Attendees who li...","From people with a ""biplane"" in their profile description find those who attended an event once and liked a post that can be said to be a positive one"


In [31]:
widget.get_changed_df().to_csv('new_splitted.csv')